#### Intervention keys

In [3]:
import pandas as pd
import random
import numpy as np

In [1]:
# General config & paths

from pathlib import Path

# Project root = folder where this notebook lives
PROJECT_ROOT = Path().resolve()

# Data folders (relative paths)
DATA_DIR = PROJECT_ROOT / "data"
QUALTRICS_DIR = DATA_DIR / "qualtrics"
WAVE5_DIR = QUALTRICS_DIR / "wave5"
KEYS_DIR = DATA_DIR / "keys"
MESSAGES_DIR = DATA_DIR / "messages"
AVICENNA_DIR = DATA_DIR / "avicenna"

In [23]:
parents_w5 = pd.read_csv(WAVE5_DIR / "Wave 5 - parents.csv")
date_change_w5 = pd.read_csv(WAVE5_DIR / "Wave 5 - date change.csv")
responses_w5 = pd.read_csv(WAVE5_DIR / "Wave 5 - responses.csv")

parents_w5["StartDate"] = pd.to_datetime(parents_w5["StartDate"], errors='coerce')
date_change_w5["StartDate"] = pd.to_datetime(date_change_w5["StartDate"], errors='coerce')
responses_w5["StartDate"] = pd.to_datetime(responses_w5["StartDate"], errors='coerce')
filter_date = pd.to_datetime("2025-03-01 00:00:00")
parents_w5 = parents_w5[parents_w5["StartDate"] > filter_date]
responses_w51 = responses_w5[responses_w5["StartDate"] > filter_date]
parents_w5_previous = parents_w5[parents_w5["StartDate"] < filter_date]


# Sort by 'Start Date' in descending order so the most recent dates come first
date_change_5 = date_change_w5.sort_values(by='StartDate', ascending=False)
responses_w5 = responses_w5[2:]
parents_w5 = parents_w5[2:]

print(len(parents_w5))

parents_w5["Random ID"] = parents_w5["Random ID"].astype(float)


4662


C:\Users\77197jsc\AppData\Local\Temp\ipykernel_34032\2275296245.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parents_w5["StartDate"] = pd.to_datetime(parents_w5["StartDate"], errors='coerce')
C:\Users\77197jsc\AppData\Local\Temp\ipykernel_34032\2275296245.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_change_w5["StartDate"] = pd.to_datetime(date_change_w5["StartDate"], errors='coerce')
C:\Users\77197jsc\AppData\Local\Temp\ipykernel_34032\2275296245.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  responses_w5["StartDate"] = pd.to_datetime(responses_w5["S

In [24]:
parents_w5 = parents_w5[(parents_w5['Progress'] == "100") & ((parents_w5['Informed_consent_exp'] == "I give CONSENT  for my child to participate in the Between Us research program.") | (parents_w5['Q3'] == "I give CONSENT  for my child to participate in the Between Us research program."))]
print(parents_w5["Random ID"].nunique())

851


In [25]:
parents_w5["Game_preview_exp"] = parents_w5["Game_preview_exp"].fillna(parents_w5['Game_preview_ctr'])
parents_w5['Mail_child_exp'] = parents_w5['Mail_child_exp'].fillna(parents_w5['Mail_child_ctr'])
parents_w5 = parents_w5.reset_index(drop=True)
print(parents_w5["Game_preview_exp"].isna().sum())
print(len(responses_w5))

11
1004


## Qualtrics data

In [26]:
keys_filtered = parents_w5[(parents_w5['Progress'] == "100") & ((parents_w5['Informed_consent_exp'] == "I give CONSENT  for my child to participate in the Between Us research program.") | (parents_w5['Q3'] == "I give CONSENT  for my child to participate in the Between Us research program."))]

# Add column for being in control group (0 = no, 1 = yes)
keys_filtered['control'] = keys_filtered['Q3'].apply(lambda x: 1 if x == "I give CONSENT  for my child to participate in the Between Us research program." else 0)
len(keys_filtered)

855

In [27]:
keys_filtered['control'].value_counts()

control
0    634
1    221
Name: count, dtype: int64

In [28]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

keys_filtered['Random ID'] = keys_filtered['Random ID'].astype(float)
responses_w5['Child_ID']= responses_w5['Child_ID'].astype(float)

keys_filtered['Mail_child_exp'] = keys_filtered['Mail_child_exp'].str.lower()
responses_w5['Mail_child']= responses_w5['Mail_child'].str.lower()

In [31]:
# Match post-chatgame data with parent registration data (by mail or ID)

# Ensure numeric IDs
keys_filtered["Random ID"] = pd.to_numeric(keys_filtered["Random ID"], errors="coerce")
responses_w5["Child_ID"] = pd.to_numeric(responses_w5["Child_ID"], errors="coerce")

# Keys → responses: which parents can we match?
match_by_mail_keys = keys_filtered["Mail_child_exp"].isin(responses_w5["Mail_child"])
match_by_id_keys   = keys_filtered["Random ID"].isin(responses_w5["Child_ID"])

print("Keys matched by mail:", match_by_mail_keys.sum())
print("Keys matched by ID:  ", match_by_id_keys.sum())

filtered_keys = keys_filtered[match_by_mail_keys | match_by_id_keys]
print("Total matched key rows:", len(filtered_keys))

# Responses → keys: which children can't be matched at all?
match_by_mail_resp = responses_w5["Mail_child"].isin(keys_filtered["Mail_child_exp"])
match_by_id_resp   = responses_w5["Child_ID"].isin(keys_filtered["Random ID"])

neither_mask = ~(match_by_mail_resp | match_by_id_resp)
neither = responses_w5[neither_mask].copy()

print("Unmatched response rows:", len(neither))

# Keep only "problem children" with Progress > 60
neither["Progress"] = pd.to_numeric(neither["Progress"], errors="coerce")
neither = neither[neither["Progress"] > 60]

problem_children = list(zip(neither["Mail_child"], neither["Child_ID"]))
problem_childs = "; ".join(neither["Mail_child"].astype(str))

Keys matched by mail: 235
Keys matched by ID:   305
Total matched key rows: 309
Unmatched response rows: 684


In [33]:
problem_children[:5]

[('romeobenson23@gmail.com', 43876.0),
 ('gaagoobilo@gmail.com', 64938.0),
 ('likohgre@gmail.com', 18557.0),
 ('gizmo910@icloud.com', 28371.0),
 ('irishvader@outlook.com', 97077.0)]

In [34]:
filtered_keys.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Q_RecaptchaScore,Q_RelevantIDDuplicate,Q_RelevantIDDuplicateScore,Q_RelevantIDFraudScore,Q_RelevantIDLastStartDate,Country_of_residence,Country_of_residence_6_TEXT,Children,Children_age,Dietary_identity,Dietary_identity_6_TEXT,Lunch_habit,Lunch_availability,Q3,Mail_parent_ctr,Mail_child_ctr,Research_update_ctr,Game_preview_ctr,Informed_consent_exp,Mail_parent_exp,Mail_child_exp,Research_update_exp,Game_preview_exp,Children_age_years,Birthplace_parents,Ethnic_group,Ethnic_group_5_TEXT,SES_education,SES_income_UK,SES_income_ROI,Random ID,id,control
0,2025-03-10 08:24:00,2025-03-10 08:25:14,Survey Preview,NaN,100,73,True,2025-03-10 08:25:15,R_8P6HPEwXEQWZHrw,NaN,NaN,NaN,NaN,52.3824,4.8995,preview,EN,1,true,100,0,2025-03-10 14:23:24,Scotland,NaN,Yes,11-16 years-old,Flexitarian (Only rarely eating meat),NaN,My child always gets food at school.,Yes,I give CONSENT for my child to participate in...,j.schlicht@t-online.de,j.schlicht@t-online.de,NaN,"Thursday, 20th March at 7 PM",NaN,NaN,j.schlicht@t-online.de,NaN,"Thursday, 20th March at 7 PM",15 years old,Yes,Mixed or Multiple,NaN,Higher or secondary or further education (A-le...,>£ 100K,NaN,73229.0,NaN,1
14,2025-03-12 11:17:27,2025-03-12 11:26:13,IP Address,80.233.72.212,100,526,True,2025-03-12 11:26:14,R_8rALJn8B0LlbjbB,NaN,NaN,NaN,NaN,53.3382,-6.2591,anonymous,EN,0.8999999761581421,NaN,0,0,NaN,Republic of Ireland,NaN,Yes,"0-4 years-old,11-16 years-old",Omnivore (Eating all animal products),NaN,My child always brings food from home or a store.,Yes,I give CONSENT for my child to participate in...,Oluwaseyikole@gmail.com,Oluwaseyikoleoluwa@gmail.com,I would like to be informed by email about the...,"Monday, 17th March at 7 PM",NaN,NaN,oluwaseyikoleoluwa@gmail.com,NaN,"Monday, 17th March at 7 PM",14 years old,Yes,"Black, Black British, Black Welsh, Black Irish...",NaN,Post-graduate degree,NaN,Prefer not to say,73764.0,QGYe2jXkvVjAgrXp,1
21,2025-03-12 11:29:03,2025-03-12 11:36:42,IP Address,37.153.153.4,100,459,True,2025-03-12 11:36:43,R_2rXLqTkZZ2A0VRn,NaN,NaN,NaN,NaN,52.4064,-1.5082,anonymous,EN,0.8999999761581421,NaN,0,10,NaN,England,NaN,Yes,"0-4 years-old,11-16 years-old",Omnivore (Eating all animal products),NaN,My child always brings food from home or a store.,Yes,I give CONSENT for my child to participate in...,Alabitemitayo6@gmail.com,Alabitemitayo6@gmail.com,I would like to be informed by email about the...,"Monday, 17th March at 7 PM",NaN,NaN,alabitemitayo6@gmail.com,NaN,"Monday, 17th March at 7 PM",14 years old,No,NaN,NaN,Post-graduate degree,£ 71-100K,NaN,35302.0,BdVOAGxW7z5o68JL,1
24,2025-03-12 11:34:40,2025-03-12 11:39:21,IP Address,86.28.228.224,100,280,True,2025-03-12 11:39:21,R_2E6pv49uDd4HWn8,NaN,NaN,NaN,NaN,51.9045,-0.1984,anonymous,EN,1,NaN,0,0,NaN,England,NaN,Yes,"11-16 years-old,17- 22 years-old",Omnivore (Eating all animal products),NaN,My child sometimes brings food from home/a sto...,Yes,NaN,NaN,NaN,NaN,NaN,I give CONSENT for my child to participate in...,charlottepayne213@yahoo.com,charpayinbox283@outlook.com,NaN,"Tuesday, 18th March at 7 PM",14 years old,No,NaN,NaN,Secondary school,£ 20-50K,NaN,45039.0,GlMq9eb14BDADpXY,0
27,2025-03-12 11:39:18,2025-03-12 11:45:10,IP Address,86.172.89.142,100,351,True,2025-03-12 11:45:11,R_22JECgDBg41Wezs,NaN,NaN,NaN,NaN,51.358,-0.8521,anonymous,EN,0.8999999761581421,NaN,0,0,NaN,England,NaN,Yes,"5-10 years-old,11-16 years-old",Omnivore (Eating all animal products),NaN,My child sometimes brings food from home/a sto...,Yes,NaN,NaN,NaN,NaN,NaN,I give CONSENT for my child to participate in...,mrazzad@hotmail.com,mrazzad@hotmail.com,NaN,"Monday, 17th March at 7 PM",15 years old,No,NaN,NaN,College or university,£ 71-100K,NaN,19873.0,vGWjoqE7zjlozDOX,0


In [35]:
chatgame_w5 = filtered_keys.drop_duplicates(subset=["id"])
chatgame_w5 = chatgame_w5["id"]

print(len(chatgame_w5))

308


In [36]:
#chatgame_w5.to_csv("chatgame_w5.csv")

## Assingning arms

In [37]:
# Assign a random number (arms 1-4) for people that are not in the control group
filtered_keys['condition'] = filtered_keys['control'].apply(lambda x: random.randint(2, 4) if x == 0 else 1)

C:\Users\77197jsc\AppData\Local\Temp\ipykernel_34032\3742310706.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_keys['condition'] = filtered_keys['control'].apply(lambda x: random.randint(2, 4) if x == 0 else 1)


In [38]:
# Value 4 needs to be split up into two group 4 and 5 which assigns the order of tailored/non-tailored intervention

def random_replace(value):
    if value == 4:
        return np.random.choice([4, 5])
    return value

# Apply the function to the column
filtered_keys['condition'] = filtered_keys['condition'].apply(random_replace)

C:\Users\77197jsc\AppData\Local\Temp\ipykernel_34032\2709943707.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_keys['condition'] = filtered_keys['condition'].apply(random_replace)


In [39]:
filtered_keys = filtered_keys[['Mail_child_exp', 'Mail_child_ctr', 'id', 'Random ID', 'condition']]

filtered_keys = filtered_keys.rename(columns={"id": "Norstat ID", "Random ID": "Movez ID"})

#Collapse mails into a single column
filtered_keys['Mail_child_exp'] = filtered_keys['Mail_child_exp'].fillna(filtered_keys['Mail_child_ctr'])

In [40]:
filtered_keys_w5 = filtered_keys.drop(columns = "Mail_child_ctr")

filtered_keys_w5 = filtered_keys_w5.reset_index()

In [41]:
filtered_keys_w5["Movez ID"] = filtered_keys_w5["Movez ID"].astype(float)

In [42]:
sorted_keys_w5 = filtered_keys_w5.sort_values(by="Movez ID", ascending=True)

sorted_keys_w5

,index,Mail_child_exp,Norstat ID,Movez ID,condition
163,408,adamprivy98@gmail.com,NaBzAByMyeE9YyVM,10207.0,2
283,761,preciousfunmilayo2011@gmail.com,vGWjoqjkaEv2zDOX,10207.0,1
274,747,jamesryan8511@gmail.com,5YwxodjeQ159Oylz,10786.0,3
219,571,phebeechimezie@gmail.com,5Ywxodbd8QX9Oylz,11859.0,4
279,757,debugu737@outlook.com,D5je9bPYMPDoRMYl,11861.0,1
10,42,dongyuewen@gmail.com,NaBzAB8WxN02YyVM,12311.0,1
151,383,myslimeandi@gmail.com,j1Le2D6Mg8RAxEOy,13379.0,1
83,234,suzanacrd@gmail.com,RWLV2Zy4mpPo5vN3,13427.0,4
265,724,ericandydrew08@gmail.com,81K7AmvLv4xoZmyO,13829.0,1
114,291,chloewebb7y7@gmail.com,JkVd2ngPqE69nLDq,13847.0,2


In [43]:
duplicated_keys = filtered_keys_w5[filtered_keys_w5["Movez ID"].duplicated()]

len(sorted_keys_w5)

309

In [44]:
filtered_keys_w5["condition"].value_counts()

condition
1    78
2    77
3    70
4    43
5    41
Name: count, dtype: int64

In [252]:
#filtered_keys_w5.to_csv(WAVE5_DIR / 'qualtrics_keys_w5.csv', index=False)

### Avicenna keys

In [47]:
qualtrics_keys_w5 = pd.read_csv(KEYS_DIR / "wave5" /  "qualtrics_keys_w5.csv")

# Loading the Avicenna participation file
avicenna = pd.read_csv(AVICENNA_DIR / "wave5" / 'Wave 5 - avicenna.csv')

# Remove timezone from "Scheduled Time"
avicenna["Start Time"] = pd.to_datetime(avicenna["Start Time"], errors='coerce').dt.tz_localize(None)
avicenna = avicenna[avicenna["Start Time"] > filter_date]

#avicenna2["Start Time"] = pd.to_datetime(avicenna["Start Time"], errors='coerce').dt.tz_localize(None)
#avicenna2 = avicenna[avicenna["Start Time"] > filter_date]

In [48]:
print(len(avicenna))

286


In [49]:
avicenna[["First Name", "Last Name", "Email Address"]]

,First Name,Last Name,Email Address
0,BU,12345,test@testwer.com
1,BU,61054,cad0902002@gmail.com
2,BU,56929,fname8752@gmail.com
3,BU,73047,mcalinnl@gmail.com
4,Chika,NaN,dchika008@gmail.com
5,BU,82231,annayusufgambo@gmail.com
6,BU,10786,petet1680@gmail.com
7,BU,20123,fibilriddle@gmail.com
8,BU,88415,mezzazxz2018@gmail.com
9,BU,19924,shayegreen@outlook.com


In [50]:
updates = {
    4: 24147,
    21: 80552,
    30: 24906,
    50: 65683,
    53: 24850,
    67: 99069,
    93: 97389,
    103: 52999,
    105: 87143,
    114: 38813,
    120: 79316,
    122: 69968,
    123: 77832,
    127: 42199,
    156: 60119,
    189: 80552,
    204: 76126,
    210: 25573,
    214: 19626,
    216: 47000,
    232: 99715,
    233: 79699,
}

for idx, value in updates.items():
    avicenna.at[idx, "Last Name"] = value


In [52]:
avicenna["Last Name"] = pd.to_numeric(avicenna["Last Name"], errors="coerce").fillna(avicenna["Last Name"])

filtered_subset = filtered_keys_w5[
    ~filtered_keys_w5["Mail_child_exp"].isin(avicenna["Email Address"]) &
    ~filtered_keys_w5["Movez ID"].isin(avicenna["Last Name"])
]

In [53]:
# Selecting relevant columns
avicenna = avicenna[["ID", "Last Name", "Email Address"]]

avicenna = avicenna.rename(columns={"Last Name": "Movez ID", "Email Address": "Mail_child_exp"})


# Only selecting particiaptions that have a valid 5-digit ID
avicenna['Movez ID'] = pd.to_numeric(avicenna['Movez ID'], errors='coerce').fillna(0)

#avicenna = avicenna[8:]
print(len(avicenna))

286


In [54]:
avicenna

,ID,Movez ID,Mail_child_exp
0,115776,12345.0,test@testwer.com
1,115763,61054.0,cad0902002@gmail.com
2,115754,56929.0,fname8752@gmail.com
3,115753,73047.0,mcalinnl@gmail.com
4,114865,24147.0,dchika008@gmail.com
5,115750,82231.0,annayusufgambo@gmail.com
6,115748,10786.0,petet1680@gmail.com
7,115743,20123.0,fibilriddle@gmail.com
8,115744,88415.0,mezzazxz2018@gmail.com
9,115740,19924.0,shayegreen@outlook.com


In [55]:
# Convert "Movez ID" to string in both DataFrames to avoid merge type issues
avicenna["Movez ID"] = avicenna["Movez ID"].astype(str)
qualtrics_keys_w5["Movez ID"] = qualtrics_keys_w5["Movez ID"].astype(str)

# Merge on "Mail_child_exp" first
merged1 = avicenna.merge(qualtrics_keys_w5, on="Mail_child_exp", how="inner")

# Find unmatched avicenna rows
unmatched_avicenna = avicenna[~avicenna["Mail_child_exp"].isin(merged1["Mail_child_exp"])]

# Merge the remaining unmatched rows on "Movez ID"
merged2 = unmatched_avicenna.merge(qualtrics_keys_w5, on="Movez ID", how="inner")

# Find still unmatched avicenna rows
still_unmatched_avicenna = unmatched_avicenna[~unmatched_avicenna["Movez ID"].isin(merged2["Movez ID"])]

# Combine all merged and unmatched rows
final_merged = pd.concat([merged1, merged2, still_unmatched_avicenna], ignore_index=True)

# Display result
final_merged

final_merged["Movez ID_x"] = final_merged["Movez ID_x"].astype(float).replace(0, pd.NA)  # Convert 0 to NaN
final_merged["Movez ID_x"].fillna(final_merged["Movez ID_y"], inplace=True)
final_merged["Movez ID_x"].fillna(final_merged["Movez ID"], inplace=True)

final_merged["Mail_child_exp"].fillna(final_merged["Mail_child_exp_x"], inplace=True)

final_merged.drop(columns=["Movez ID", "Mail_child_exp_x", "Mail_child_exp_y"], inplace=True)

C:\Users\77197jsc\AppData\Local\Temp\ipykernel_34032\1455405586.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_merged["Movez ID_x"].fillna(final_merged["Movez ID_y"], inplace=True)
C:\Users\77197jsc\AppData\Local\Temp\ipykernel_34032\1455405586.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan n

In [56]:
len(final_merged)

288

In [58]:
final_merged.at[286, "Norstat ID"] = "kqPlARXJzKg2WndY"
final_merged.at[286, "condition"] = 4.0
final_merged.at[285, "Norstat ID"] ="jnWOo0R5nzb9Xzex"
final_merged.at[285, "condition"] = 1.0

#Change condition bc. TST was not reliable
final_merged.at[24, "condition"] = 2.0 # little chats
final_merged.at[66, "condition"] = 2.0 # little chats

final_merged = final_merged.drop(59)
final_merged = final_merged.drop(101)
final_merged.at[115, "condition"] = 4.0

9

In [59]:
final_merged["Norstat ID"].duplicated().sum()

duplicates = final_merged[final_merged["Norstat ID"].duplicated(keep=False)]

In [60]:
#Lets have a look at duplicated items
duplicates

,ID,Movez ID_x,Mail_child_exp,index,Norstat ID,Movez ID_y,condition
39,115519,69968.0,kipkiruikoechrantich3@gmail.com,1959.0,nKY7owXjYk82G3v4,69968.0,2.0
68,115185,52999.0,qmazeedah@gmail.com,2168.0,gJdBAEy7naJAbz5P,52999.0,5.0
85,115114,77832.0,runicekhan@gmail.com,1960.0,p3MN234kmm19l1ag,77832.0,4.0
96,115037,46857.0,icelinisaac1@gmail.com,1720.0,Vj6N98XWmGN98drl,46857.0,4.0
115,114918,22915.0,icelinisaac2@gmail.com,1801.0,Vj6N98XWmGN98drl,22915.0,4.0
124,114887,80552.0,roangosling30@gmail.com,1539.0,GLPp2O5G7glA03ym,80552.0,3.0
136,114816,19626.0,marcuseriksen50@gmail.com,185.0,5nJ4ApEKnb4oG3kx,19626.0,1.0
191,115732,80552.0,scronin11@hotmail.com,1539.0,GLPp2O5G7glA03ym,NaN,3.0
192,115730,52999.0,omoba.quadri12@gmail.com,2168.0,gJdBAEy7naJAbz5P,NaN,5.0
193,115698,80552.0,niallgosling1@gmail.com,1539.0,GLPp2O5G7glA03ym,NaN,3.0


In [61]:
keys_w5 = final_merged
keys_w5["ID"] = keys_w5["ID"].astype(float)

In [63]:
keys_w5["Movez ID_x"] = keys_w5["Movez ID_x"].astype(float)
#keys_w5 = keys_w5.drop_duplicates(subset=["Movez ID_x"], keep="last")

In [64]:
keys_w5

,ID,Movez ID_x,Mail_child_exp,index,Norstat ID,Movez ID_y,condition
0,115763.0,61054.0,cad0902002@gmail.com,2243.0,3YVN2MyPJaV2bRn5,61054.0,3.0
1,115753.0,73047.0,mcalinnl@gmail.com,1820.0,kqPlARXJNDB2WndY,73047.0,5.0
2,114865.0,24147.0,dchika008@gmail.com,1882.0,JkVd2nax4VgonLDq,24147.0,5.0
3,115743.0,20123.0,fibilriddle@gmail.com,2102.0,5YwxodY7xzM2Oylz,20123.0,4.0
4,115744.0,88415.0,mezzazxz2018@gmail.com,1889.0,nKY7owO5gDeoG3v4,88415.0,1.0
5,115740.0,19924.0,shayegreen@outlook.com,2035.0,gJdBAEDRZNZ9bz5P,19924.0,2.0
6,115738.0,59564.0,sneakytimbler@gmail.com,2099.0,zExm2JbDLay26PyK,59564.0,1.0
7,115737.0,93476.0,marcfree39@gmail.com,1995.0,gJdBAE475RK9bz5P,93476.0,2.0
8,115735.0,77473.0,deadlydash@hotmail.com,2097.0,vGWjoqMgmbRozDOX,77473.0,1.0
9,115733.0,83271.0,michaelpearl380@gmail.com,1588.0,GlMq9ePWyLY2DpXY,83271.0,3.0


In [65]:
keys_w5 = keys_w5.rename(columns={"Movez ID_x": "Movez ID"})

In [66]:
keys_small = keys_w5[["ID", "condition"]]

In [67]:
#keys_small.to_csv(KEYS_DIR / 'wave5' / "keys_w5_small.csv", index = False)
#keys_w5.to_csv(KEYS_DIR / 'wave5' /"keys_w5_new.csv", index = False)

## Reponse rate

In [5]:
import pandas as pd

In [68]:
response_w5 = pd.read_csv(AVICENNA_DIR / 'wave5'/ 'study-3843-activity-sessions-2025-04-15-12-04-05.csv')
response_w5 = response_w5[["Activity ID", "User ID", "Scheduled Time", "Status"]]
keys_w5 = pd.read_csv(KEYS_DIR / "keys_w5.csv")

response_w5["Scheduled Time"] = pd.to_datetime(response_w5["Scheduled Time"], errors='coerce')
# Define the date range
start_date = '2025-03-01'
end_date = '2025-04-25'

response_w5 = response_w5[
    (response_w5['Scheduled Time'].isna()) | 
    ((response_w5['Scheduled Time'] >= start_date) & (response_w5['Scheduled Time'] <= end_date))
]

In [69]:
response_w5 = response_w5.merge(keys_w5, left_on = "User ID", right_on="ID", how= "left")

In [70]:
response_w5["Status"] = response_w5['Status'].replace({
    'Completed': 1,
    'Unknown': 99,
    'Expired': 0,
    'Canceled': 99,
    'Blocked': 99,
     'NaN': 99})

C:\Users\77197jsc\AppData\Local\Temp\ipykernel_34032\3837578165.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  response_w5["Status"] = response_w5['Status'].replace({


In [71]:
pd.set_option('display.max_rows', None)

In [72]:
# Ensure Status is numeric
response_w5["Status"] = response_w5["Status"].astype(int)

# Sort so that Status==1 comes last in case of tie-break
response_w5 = response_w5.sort_values(by=["User ID", "Activity ID", "Status"])

# Define the function to resolve duplicates
def resolve_duplicates(group):
    status_1_rows = group[group["Status"] == 1]
    if len(status_1_rows) > 0:
        return status_1_rows.iloc[[0]]  # Keep the FIRST row with Status == 1
    else:
        return group.iloc[[-1]]  # Keep last row if no Status == 1 found

# Apply it
deduped = (
    response_w5
    .groupby(["User ID", "Activity ID"], group_keys=False)
    .apply(resolve_duplicates)
    .reset_index(drop=True)
)

C:\Users\77197jsc\AppData\Local\Temp\ipykernel_34032\1328757805.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(resolve_duplicates)


In [73]:
deduped = deduped[deduped["Movez ID"].notna()]

In [74]:
# Sort the dataframe so we can handle duplicates effectively
response_w5 = response_w5.sort_values(by=["User ID", "Activity ID", "Status"])

# Define a function to handle duplicates
def resolve_duplicates(group):
    if (group["Status"] == 1).any():  # If any row has Status == 1
        return group[group["Status"] == 1].iloc[[-1]]  # Keep the last row with Status == 1
    else:
        return group.iloc[[-1]]  # Otherwise, keep the last row

# Group by the duplicated columns and resolve duplicates
response_w5 = response_w5.groupby(["User ID", "Activity ID"], group_keys=False).apply(resolve_duplicates)

C:\Users\77197jsc\AppData\Local\Temp\ipykernel_34032\1666720477.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  response_w5 = response_w5.groupby(["User ID", "Activity ID"], group_keys=False).apply(resolve_duplicates)


In [75]:
# Pivot the dataframe
response_w5_wide = deduped.pivot(index='User ID', columns='Activity ID', values='Status')

response_w5_wide = response_w5_wide.reset_index()
response_w5_wide = response_w5_wide.merge(keys_w5[["ID", "Norstat ID", "condition"]], left_on="User ID", right_on="ID", how ="left")

In [77]:
response_w5_wide.head()

,User ID,22395,22404,22405,22406,22407,22408,22409,22410,22411,22412,22414,22415,22419,22596,22597,22606,22607,22608,22609,22610,22611,22612,22613,22614,22716,ID,Norstat ID,condition
0,114514,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,114514.0,LDen9K6mqnQ2ElVw,3.0
1,114719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,114719.0,vGWjoqeXrWBozDOX,1.0
2,114720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,114720.0,p3MN23ZRzaBol1ag,1.0
3,114721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,114721.0,NaBzAB0rea32YyVM,1.0
4,114722,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,114722.0,LKkR2Xm4lVz9ZaYV,4.0


In [78]:
response_w5_wide_ctr = response_w5_wide[response_w5_wide["condition"] == 1.0]
response_w5_wide_exp = response_w5_wide[response_w5_wide["condition"] != 1.0]

In [79]:
print(len(response_w5_wide_exp))

209


In [80]:
response_w5_wide_ctr.head()

,User ID,22395,22404,22405,22406,22407,22408,22409,22410,22411,22412,22414,22415,22419,22596,22597,22606,22607,22608,22609,22610,22611,22612,22613,22614,22716,ID,Norstat ID,condition
1,114719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,114719.0,vGWjoqeXrWBozDOX,1.0
2,114720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,114720.0,p3MN23ZRzaBol1ag,1.0
3,114721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,114721.0,NaBzAB0rea32YyVM,1.0
6,114724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,114724.0,Vw142rv6Gak2aNPX,1.0
12,114732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,114732.0,qRZK2gvvelYobdka,1.0


In [81]:
pairs = [
    ([22395, 22716], 'Day 1'),
    ([22606, 22404], 'Day 2'),
    ([22607, 22405], 'Day 3'),
    ([22608, 22406], 'Day 4'),
    ([22609, 22407], 'Day 5'),
    ([22610, 22408], 'Day 6'),
    ([22611, 22409], 'Day 7'),
    ([22612, 22410], 'Day 8'),
    ([22613, 22411], 'Day 9'),
    ([22614, 22412], 'Day 10',),
]

In [82]:
#for columns, day_name in pairs:
    #pilot_resp_wide[day_name] = ((pilot_resp_wide[columns[0]] == 1) & (pilot_resp_wide[columns[1]] == 1)).astype(int)

for columns, day_name in pairs:
    response_w5_wide[day_name] = (
        ((response_w5_wide['condition'] == 1) & (response_w5_wide[columns[0]] == 1)) |
        ((response_w5_wide['condition'] != 1) & (response_w5_wide[columns[0]] == 1) & (response_w5_wide[columns[1]] == 1))
    ).astype(int)

response_w5_wide = response_w5_wide.rename(columns={22414: "Baseline", 22415: "Weekend 1", 22419: "Weekend 2"})

In [163]:
column_to_front = "condition"
cols = [column_to_front] + [col for col in response_w5_wide.columns if col != column_to_front]
response_w5_wide = response_w5_wide[cols]

response_w5_wide.head()

,condition,User ID,22395,22404,22405,22406,22407,22408,22409,22410,22411,22412,Baseline,Weekend 1,Weekend 2,22596,22597,22606,22607,22608,22609,22610,22611,22612,22613,22614,22716,ID,Norstat ID,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10
0,3.0,114514,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,114514.0,LDen9K6mqnQ2ElVw,1,1,1,1,1,1,1,1,1,1
1,1.0,114719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,114719.0,vGWjoqeXrWBozDOX,0,1,1,1,1,1,1,1,1,1
2,1.0,114720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,114720.0,p3MN23ZRzaBol1ag,0,1,0,1,0,1,1,1,1,1
3,1.0,114721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,114721.0,NaBzAB0rea32YyVM,0,1,1,1,1,1,1,1,1,1
4,4.0,114722,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,114722.0,LKkR2Xm4lVz9ZaYV,0,1,1,1,1,1,1,0,0,1


In [164]:

w5_ids = list(keys_w5["ID"])

response_w5 = response_w5_wide[response_w5_wide["User ID"].isin(w5_ids)]

response_w5 = response_w5[["condition", "User ID", "Baseline", "Day 1", "Day 2", "Day 3", "Day 4", "Day 5", "Weekend 1", "Day 6", "Day 7", "Day 8", "Day 9", "Day 10", "Weekend 2"]]

response_w5.head()

,condition,User ID,Baseline,Day 1,Day 2,Day 3,Day 4,Day 5,Weekend 1,Day 6,Day 7,Day 8,Day 9,Day 10,Weekend 2
0,3.0,114514,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0
1,1.0,114719,1.0,0,1,1,1,1,1.0,1,1,1,1,1,1.0
2,1.0,114720,1.0,0,1,0,1,0,1.0,1,1,1,1,1,1.0
3,1.0,114721,1.0,0,1,1,1,1,1.0,1,1,1,1,1,1.0
4,4.0,114722,1.0,0,1,1,1,1,1.0,1,1,0,0,1,1.0


In [87]:
drop = [115776]

#afternoon_children = [108266, 108279, 108339, 108359, 108384, 108386, 108400, 108694,108697, 108754, 108760, 108762, 108763, 108764, 108773, 108787, 108792, 108900]

In [88]:
response_w5["User ID"] = response_w5["User ID"].astype(int)

# Drop rows where User ID is in the drop list
response_w5_new = response_w5[~response_w5["User ID"].isin(drop)]

# Reset the index (optional, for cleaner indexing)
response_w5_new = response_w5_new.reset_index(drop=True)

In [95]:
# Set pandas display option to show all columns
pd.set_option('display.max_rows', None)
response_w5_new.head()

,condition,User ID,Baseline,Day 1,Day 2,Day 3,Day 4,Day 5,Weekend 1,Day 6,Day 7,Day 8,Day 9,Day 10,Weekend 2
0,3.0,114514,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0
1,1.0,114719,1.0,0,1,1,1,1,1.0,1,1,1,1,1,1.0
2,1.0,114720,1.0,0,1,0,1,0,1.0,1,1,1,1,1,1.0
3,1.0,114721,1.0,0,1,1,1,1,1.0,1,1,1,1,1,1.0
4,4.0,114722,1.0,0,1,1,1,1,1.0,1,1,0,0,1,1.0


In [90]:
wave5_resp_wide = response_w5_new.merge(keys_w5[["ID", "Norstat ID", "condition", "Mail_child_exp"]], left_on="User ID", right_on="ID", how ="left")

In [96]:
wave5_resp_wide.head()

,condition,User ID,Baseline,Day 1,Day 2,Day 3,Day 4,Day 5,Weekend 1,Day 6,Day 7,Day 8,Day 9,Day 10,Weekend 2,ID,Norstat ID
0,3.0,114514,1.0,1,1,1,1,1,1.0,1,1,1,1,1,1.0,114514.0,LDen9K6mqnQ2ElVw
1,1.0,114719,1.0,0,1,1,1,1,1.0,1,1,1,1,1,1.0,114719.0,vGWjoqeXrWBozDOX
2,1.0,114720,1.0,0,1,0,1,0,1.0,1,1,1,1,1,1.0,114720.0,p3MN23ZRzaBol1ag
3,1.0,114721,1.0,0,1,1,1,1,1.0,1,1,1,1,1,1.0,114721.0,NaBzAB0rea32YyVM
4,4.0,114722,1.0,0,1,1,1,1,1.0,1,1,0,0,1,1.0,114722.0,LKkR2Xm4lVz9ZaYV


In [97]:
#Some adjustments before saving

wave5_resp_wide = wave5_resp_wide.drop(columns=['condition_y', 'Mail_child_exp'])

wave5_resp_wide.fillna(99, inplace=True)

wave5_resp_wide = wave5_resp_wide.rename(columns={"condition_x": "condition"})

KeyError: "['condition_y', 'Mail_child_exp'] not found in axis"

In [98]:
wave5_resp_wide["Norstat ID"].duplicated().sum()

8

In [99]:
wave5_resp_wide_small = wave5_resp_wide.drop(columns=["ID", "condition_y", "Mail_child_exp"])

KeyError: "['condition_y', 'Mail_child_exp'] not found in axis"

In [39]:
#wave5_resp_wide_small.to_csv(DATA_DIR / "processed" / "wave5" / "Wave 5 - full responses.csv")

In [100]:
#wave5 = pd.read_csv("Wave 5 - full responses.csv")

## Importing Mistral TST data

In [103]:
#Importing the mistral data
mistral_w51 = pd.read_csv(MESSAGES_DIR/ 'wave5'/'day1_0_mistral-large-2407_0.7_all_shots_parsed.csv')
mistral_w52 = pd.read_csv(MESSAGES_DIR/ 'wave5'/'day2_0_mistral-large-2407_0.7_all_shots_parsed.csv')
mistral_w53 = pd.read_csv(MESSAGES_DIR/ 'wave5'/'day3_0_mistral-large-2407_0.7_all_shots_parsed.csv')
mistral_w54 = pd.read_csv(MESSAGES_DIR/ 'wave5'/'day4_0_mistral-large-2407_0.7_all_shots_parsed.csv')
mistral_w55 = pd.read_csv(MESSAGES_DIR/ 'wave5'/'day5_0_mistral-large-2407_0.7_all_shots_parsed.csv')
mistral_w56 = pd.read_csv(MESSAGES_DIR/ 'wave5'/'day6_0_mistral-large-2407_0.7_all_shots_parsed.csv')
mistral_w57 = pd.read_csv(MESSAGES_DIR/ 'wave5'/'day7_0_mistral-large-2407_0.7_all_shots_parsed.csv')
mistral_w58 = pd.read_csv(MESSAGES_DIR/ 'wave5'/'day8_0_mistral-large-2407_0.7_all_shots_parsed.csv')

print(len(mistral_w51))
print(len(mistral_w52))
print(len(mistral_w53))
print(len(mistral_w54))
print(len(mistral_w55))
print(len(mistral_w56))
print(len(mistral_w57))
print(len(mistral_w58))

1488
732
432
300
264
432
408
72


In [104]:
mistral_w5 = pd.concat([mistral_w51, mistral_w52, mistral_w53, mistral_w54, mistral_w55, mistral_w56, mistral_w57, mistral_w58], ignore_index=True)

In [105]:
mistral_w5.head()

,prompt_id,model,temperature,shots,user_id,message_id,neutral_message,tst_message,tst_explanation,tst_id,llm_tst
0,0,mistral-large-2407,0.7,all,13427,0,"Hello! I’m Sasha, nice to meet you! Over the n...","Hey! I’m Sasha, nice to meet you! So, over the...",Person X uses an informal and conversational t...,1342720250321172922,"{\n ""rewritten_sentence"": ""Hey! I’m Sasha, ni..."
1,0,mistral-large-2407,0.7,all,13427,1,Hey! Thanks for following along for the first ...,Hey! Thanks for following along for the first ...,I maintained the informal tone and used contra...,1342720250321172929,"{\n ""rewritten_sentence"": ""Hey! Thanks for fo..."
2,0,mistral-large-2407,0.7,all,13427,2,Reducing how much meat we eat can help save a ...,"Reducing meat we eat can save CO2, helps with ...",I've adapted the text to match person X's conv...,1342720250321172936,"{\n ""rewritten_sentence"": ""Reducing meat we e..."
3,0,mistral-large-2407,0.7,all,13427,3,"If everyone ate less animal foods, the CO2 fro...","If everyone ate less animal foods, CO2 from fo...","I've adopted person X's informal tone, use of ...",1342720250321172947,"{\n ""rewritten_sentence"": ""If everyone ate le..."
4,0,mistral-large-2407,0.7,all,13427,4,Animal farming is responsible for 80% of the l...,"Animal farming is bad for Amazon, like 80% of ...","I've incorporated the informal tone, use of fi...",1342720250321172952,"{\n ""rewritten_sentence"": ""Animal farming is ..."


In [203]:
#mistral_w5.to_csv("mistral_w5.csv")

In [106]:
twelve_messages = mistral_w5[:12]

In [107]:
unmatched_ids = keys_w5[~keys_w5["Movez ID"].astype(float).isin(mistral_w5["user_id"].astype(float))]
len(unmatched_ids)

In [108]:
unmatched_ids["Movez ID"] = unmatched_ids["Movez ID"].astype(float)

C:\Users\77197jsc\AppData\Local\Temp\ipykernel_34032\2839133375.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_ids["Movez ID"] = unmatched_ids["Movez ID"].astype(float)


In [109]:
unmatched_ids = unmatched_ids.sort_values(by="Movez ID", ascending=True)

In [110]:
unmatched_ids2 = unmatched_ids[(unmatched_ids["condition"] != 1.0) & (unmatched_ids["condition"] != 2.0)]

In [111]:
unmatched_ids2["condition"] = 2.0

C:\Users\77197jsc\AppData\Local\Temp\ipykernel_34032\3646411257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmatched_ids2["condition"] = 2.0


In [112]:
unmatched_ids2

,ID,Movez ID,Mail_child_exp,index,Norstat ID,Movez ID_y,condition
203,115567.0,15293.0,caropesh67@gmail.com,1612.0,JLQkAYVXkLZoOeDG,NaN,2.0
44,115341.0,33519.0,emube3vliva4yibjj@outlook.com,2235.0,jYWQA6pNmYg2PXKz,33519.0,2.0
42,115356.0,71441.0,dikefavour52@gmail.com,1781.0,lzD49L46dq493Vr8,71441.0,2.0
210,115278.0,72735.0,liiluwa1@gmail.com,916.0,ybDaoyXdpxdAQdvY,NaN,2.0
46,115323.0,73532.0,egltg7282wqh@hotmail.com,2229.0,j1Le2D6mOGkAxEOy,73532.0,2.0
83,115118.0,79316.0,queenhopen@gmail.com,2084.0,5Ywxodba0R39Oylz,58855.0,2.0
194,115686.0,91663.0,apomaamai@yahoo.com,420.0,beaJAWrXNLn9vw16,NaN,2.0
196,115684.0,99276.0,robertwinters272@yahoo.com,399.0,5Ywxodbjjav9Oylz,NaN,2.0


In [113]:
# Create a list to store all the duplicated messages
duplicated_messages = []

# Loop through unmatched_ids2 and duplicate twelve_messages for each row
for session_id in unmatched_ids2["Movez ID"]:
    # Make a copy of twelve_messages and set session_id
    temp = twelve_messages.copy()
    temp["user_id"] = session_id
    duplicated_messages.append(temp)

# Concatenate all the duplicated blocks into a single DataFrame
twelve_messages = pd.concat(duplicated_messages, ignore_index=True)

In [114]:
mistral_w5 = pd.concat([mistral_w5, twelve_messages], ignore_index=True)

In [115]:
# Create a list to store all the duplicated messages
duplicated_messages = []

# Loop through unmatched_ids2 and duplicate twelve_messages for each row
for session_id in unmatched_ids["Movez ID"]:
    # Make a copy of twelve_messages and set session_id
    temp = twelve_messages.copy()
    temp["user_id"] = session_id
    duplicated_messages.append(temp)

# Concatenate all the duplicated blocks into a single DataFrame
twelve_messages = pd.concat(duplicated_messages, ignore_index=True)

In [116]:
mistral_w5 = pd.concat([mistral_w5, twelve_messages], ignore_index=True)

In [117]:
#unmatched_ids.to_csv("unmatched-ids.csv")

In [118]:
# Convert unmatched_ids["Movez ID"] to float
unmatched_ids_float = unmatched_ids["Movez ID"].astype(float)

# Filter respones_w5 where "Child_ID" is in the list of unmatched Movez IDs
filtered_rows = responses_w5[responses_w5["Child_ID"].isin(unmatched_ids_float)]

In [120]:
date = filtered_rows[["StartDate", "Child_ID"]]

In [121]:
merge = unmatched_ids.merge(date, left_on="Movez ID", right_on="Child_ID")

In [122]:
merge = merge.sort_values(by="StartDate", ascending=True)

In [123]:
merge_small = merge[["Movez ID", "StartDate"]]

In [326]:
#merge_small.to_csv("merge_small.csv")

In [127]:
day1_stats = pd.read_csv(DATA_DIR / "chat_stats" / "W5_1 - chat_stats.csv")
day2_stats = pd.read_csv(DATA_DIR / "chat_stats" /"W5_2 - chat_stats.csv")
day3_stats = pd.read_csv(DATA_DIR / "chat_stats" /"W5_3 - chat_stats.csv")
day4_stats = pd.read_csv(DATA_DIR / "chat_stats" /"W5_4 - chat_stats.csv")
day5_stats = pd.read_csv(DATA_DIR / "chat_stats" /"W5_5 - chat_stats.csv")
day6_stats = pd.read_csv(DATA_DIR / "chat_stats" /"W5_6 - chat_stats.csv")
day7_stats = pd.read_csv(DATA_DIR / "chat_stats" /"W5_7 - chat_stats.csv")
day8_stats = pd.read_csv(DATA_DIR / "chat_stats" /"W5_8 - chat_stats.csv")

combined_stats = pd.concat([day1_stats, day2_stats, day3_stats, day4_stats, day5_stats, day6_stats, day7_stats, day8_stats], ignore_index=True)

In [128]:
combined_stats.head()

,session_id,msg_count,min_wc,max_wc,median_wc,mean_wc,se_wc,msg_count_wc_above_median,msg_count_wc_above_three
0,13427,28,1,20,5.5,6.964286,1.065925,14,21
1,13842,52,1,30,5.0,6.788462,0.763682,31,41
2,13847,14,1,9,4.0,4.142857,0.686264,8,10
3,14413,14,1,7,2.5,3.142857,0.543262,7,7
4,16527,49,1,8,3.0,2.795918,0.265633,26,26


In [125]:
mistral_w5.head(1)

,prompt_id,model,temperature,shots,user_id,message_id,neutral_message,tst_message,tst_explanation,tst_id,llm_tst
0,0,mistral-large-2407,0.7,all,13427.0,0,"Hello! I’m Sasha, nice to meet you! Over the n...","Hey! I’m Sasha, nice to meet you! So, over the...",Person X uses an informal and conversational t...,1342720250321172922,"{\n ""rewritten_sentence"": ""Hey! I’m Sasha, ni..."


In [131]:
# Count occurrences of each user_id
user_id_counts = mistral_w5["user_id"].value_counts()

# Filter user IDs that appear more than 12 times
user_ids_more_than_12 = user_id_counts[user_id_counts > 20].index

# Print the user IDs that occur more than 12 times
print(user_ids_more_than_12)

Index([15293.0, 33519.0, 71441.0, 72735.0, 73532.0, 79316.0, 91663.0, 99276.0,
       99715.0, 23496.0, 12345.0,     0.0, 81847.0, 52565.0, 58108.0, 61763.0,
       72700.0, 75200.0, 26814.0, 86696.0, 88415.0, 94448.0, 51139.0, 54911.0,
       58855.0, 83797.0, 69968.0, 10786.0, 96389.0, 77832.0, 72466.0, 21684.0,
       25007.0],
      dtype='float64', name='user_id')


In [132]:
mistral_w5 = mistral_w5.drop_duplicates(subset=["user_id", "message_id"], keep="last")

In [133]:
mistral_w5[["user_id", "message_id"]].duplicated().sum()

0

In [134]:
# Let's have a look at all the messages
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

mistral_w5["tst_message"].head()

0    Hey! I’m Sasha, nice to meet you! So, over the next two weeks, I'll be checking out healthy and sustainable eating with you. 🍎\n\nLike, a year ago, I started eating more plant-based stuff and learned a bunch along the way. It's not about being perfect; I don't think we all need to go full vegan. But, being a bit mindful of how much animal food we eat and drink can be good for our health and the planet, you know?\n\nStay tuned, 'cause I'll share lots of cool science facts and give you tips for small changes that can make a big difference. 💪
1                                            Hey! Thanks for following along for the first week. 🌱\n\nI like seeing more people choosing plant-based eating. Some of my favourite celebrities eat vegan, like Lewis Hamilton, Billie Eilish, and Ariana Grande. Others, like Harry Styles and Lionel Messi, avoid animal products for the most part.\n\nI also see new plant-based products and restaurants appearing everywhere. It looks like we'll have lots o

In [137]:
# Adjusting some columns
mistral_w5 = mistral_w5.rename(columns={"user_id": "Movez ID"})

mistral_w5["Movez ID"].value_counts().unique()

array([12], dtype=int64)

In [138]:
mistral_w5["Movez ID"] = mistral_w5["Movez ID"].astype(float)
keys_w5["Movez ID"] = keys_w5["Movez ID"].astype(float)

# Let's merge messages with the condition each person is in
mistral_w5 = keys_w5.merge(mistral_w5, on= "Movez ID", how="left")

mistral_w5.head()

,ID,Movez ID,Mail_child_exp,index,Norstat ID,Movez ID_y,condition,prompt_id,model,temperature,shots,message_id,neutral_message,tst_message,tst_explanation,tst_id,llm_tst
0,115763.0,61054.0,cad0902002@gmail.com,2243.0,3YVN2MyPJaV2bRn5,61054.0,3.0,0,mistral-large-2407,0.7,all,0,"Hello! I’m Sasha, nice to meet you! Over the next two weeks, I will explore healthy and sustainable eating with you.\n\nA year ago, I started eating more plant-based foods and learned a lot along the way. It’s not about being perfect; I don’t think we all need to go completely vegan. However, being a bit mindful of how much animal food we eat and drink can be good for our health and the planet.\n\nStay tuned, as I will share lots of interesting science facts and give you tips for small changes that can make a big difference.","hey there! so im sasha, nice to meet ya! over the next two weeks, im gonna be exploring healthy and sustainable eating with you guys.\n\nso like a year ago, i started eating more plant-based foods and learned a lot along the way. its not about being perfect, i dont think we all need to go completely vegan. but like, being a bit mindful of how much animal food we eat and drink can be good for our health and the planet, ya know?\n\nso stay tuned, im gonna share lots of interesting science facts and give you tips for small changes that can make a big difference.","Person X uses an informal tone with conversational fillers like 'so', 'ya know', 'gonna', and 'guys'. They also use informal contractions and avoid complex sentence structures. The rewritten text maintains these characteristics while keeping within the 25% length deviation limit.",6105420250325095505,"{\n ""rewritten_sentence"": ""hey there! so im sasha, nice to meet ya! over the next two weeks, im gonna be exploring healthy and sustainable eating with you guys.\n\nso like a year ago, i started eating more plant-based foods and learned a lot along the way. its not about being perfect, i dont think we all need to go completely vegan. but like, being a bit mindful of how much animal food we eat and drink can be good for our health and the planet, ya know?\n\nso stay tuned, im gonna share lots of interesting science facts and give you tips for small changes that can make a big difference."",\n ""explanation"": ""Person X uses an informal tone with conversational fillers like 'so', 'ya know', 'gonna', and 'guys'. They also use informal contractions and avoid complex sentence structures. The rewritten text maintains these characteristics while keeping within the 25% length deviation limit.""\n}"
1,115763.0,61054.0,cad0902002@gmail.com,2243.0,3YVN2MyPJaV2bRn5,61054.0,3.0,0,mistral-large-2407,0.7,all,1,"Hey! Thanks for following along for the first week.\n\nI'm excited to see how more people are choosing plant-based eating. Some of my favourite celebrities eat vegan, like Lewis Hamilton, Billie Eilish, and Ariana Grande. Others, like Harry Styles and Lionel Messi, avoid animal products for the most part.\n\nI also see new plant-based products and restaurants appearing everywhere. It looks like we'll have lots of options in the future!\n\nWhy not see if some of your favorite celebrities also eat plant-based?","Hey! Thanks for following along for the first week. \n\nSo excited to see more people choosing plant-based eating. Some of my fav celebs eat vegan, like Lewis Hamilton, Billie Eilish, and Ariana Grande. Others, like Harry Styles and Lionel Messi, avoid animal products for the most part. \n\nAlso seeing new plant-based products and restaurants everywhere. Looks like we'll have lots of options in the future! \n\nSo why not see if some of your fav celebs also eat plant-based?","I've adapted the text to match person X's conversational style by using informal language and contractions (e.g., 'So excited' instead of 'I'm excited', 'fav' instead of 'favorite'), removing some pronouns (e.g., 'Also seeing' instead of 'I also see'), and using simpler, more direct sentence structures. I'

In [170]:
#mistral_w5.to_csv("mistral_w5.csv")

In [139]:
# Remap message IDs to chronological order
id_map = {
    0: 1, 1: 7, 2: 3, 3: 9, 4: 12, 5: 6,
    6: 8, 7: 2, 8: 5, 9: 11, 10: 4, 11: 10,
}

mistral_w5["message_id"] = (
    mistral_w5["message_id"]
    .replace(id_map)
    .astype(int)
)

# Replace TST messages with neutral messages depending on condition
rules = [
    (5, 1, 6),   # condition 5 → replace messages 1–6
    (4, 7, 12),  # condition 4 → replace messages 7–12
    (2, 1, 12),  # condition 2 → replace all 1–12
]

for cond, start, end in rules:
    mask = (mistral_w5["condition"] == cond) & mistral_w5["message_id"].between(start, end)
    mistral_w5.loc[mask, "tst_message"] = mistral_w5.loc[mask, "neutral_message"]


In [140]:
mistral_w5[mistral_w5['condition'] == 4].head()

,ID,Movez ID,Mail_child_exp,index,Norstat ID,Movez ID_y,condition,prompt_id,model,temperature,shots,message_id,neutral_message,tst_message,tst_explanation,tst_id,llm_tst
36,115743.0,20123.0,fibilriddle@gmail.com,2102.0,5YwxodY7xzM2Oylz,20123.0,4.0,0,mistral-large-2407,0.7,all,1,"Hello! I’m Sasha, nice to meet you! Over the next two weeks, I will explore healthy and sustainable eating with you.\n\nA year ago, I started eating more plant-based foods and learned a lot along the way. It’s not about being perfect; I don’t think we all need to go completely vegan. However, being a bit mindful of how much animal food we eat and drink can be good for our health and the planet.\n\nStay tuned, as I will share lots of interesting science facts and give you tips for small changes that can make a big difference.","hello! i'm sasha, nice to meet you! over the next two weeks, we'll explore healthy and sustainable eating together. 😊\n\nso, a year ago, i started eating more plant-based foods and learned a lot along the way. it's not about being perfect, i don't think we all need to go completely vegan, you know? but being a bit mindful of how much animal food we eat and drink can be good for our health and the planet. 🌍\n\nstay tuned, i'll share lots of interesting science facts and give you tips for small changes that can make a big difference. 🌱","I've incorporated the informal tone and use of first-person pronouns ('i', 'we') seen in person X's style. I've also added emojis to convey emotion and tone, as person X uses them in their examples. I've used contractions (e.g., 'don't', 'you're') and simplified some sentences to maintain the conversational style. I've also used filler words like 'you know' to mimic person X's style.",2012320250322113617,"{\n ""rewritten_sentence"": ""hello! i'm sasha, nice to meet you! over the next two weeks, we'll explore healthy and sustainable eating together. 😊\n\nso, a year ago, i started eating more plant-based foods and learned a lot along the way. it's not about being perfect, i don't think we all need to go completely vegan, you know? but being a bit mindful of how much animal food we eat and drink can be good for our health and the planet. 🌍\n\nstay tuned, i'll share lots of interesting science facts and give you tips for small changes that can make a big difference. 🌱"",\n ""explanation"": ""I've incorporated the informal tone and use of first-person pronouns ('i', 'we') seen in person X's style. I've also added emojis to convey emotion and tone, as person X uses them in their examples. I've used contractions (e.g., 'don't', 'you're') and simplified some sentences to maintain the conversational style. I've also used filler words like 'you know' to mimic person X's style.""\n}"
37,115743.0,20123.0,fibilriddle@gmail.com,2102.0,5YwxodY7xzM2Oylz,20123.0,4.0,0,mistral-large-2407,0.7,all,7,"Hey! Thanks for following along for the first week.\n\nI'm excited to see how more people are choosing plant-based eating. Some of my favourite celebrities eat vegan, like Lewis Hamilton, Billie Eilish, and Ariana Grande. Others, like Harry Styles and Lionel Messi, avoid animal products for the most part.\n\nI also see new plant-based products and restaurants appearing everywhere. It looks like we'll have lots of options in the future!\n\nWhy not see if some of your favorite celebrities also eat plant-based?","Hey! Thanks for following along for the first week.\n\nI'm excited to see how more people are choosing plant-based eating. Some of my favourite celebrities eat vegan, like Lewis Hamilton, Billie Eilish, and Ariana Grande. Others, like Harry Styles and Lionel Messi, avoid animal products for the most part.\n\nI also see new plant-based products and restaurants appearing everywhere. It looks like we'll have lots of options in the future!\n\nWhy not see if some of your favorite celebrities also eat plant-based?","Person X uses lowercase 'i' for the first-person pronoun and tends to use informal language with contractio

In [142]:
# For myself (researchers) I copy messages from user 64938
#jonas_columns = mistral_w4[mistral_w4["Movez ID"] == 74256.0]
#jonas_columns["Movez ID"] = jonas_columns["Movez ID"].replace(74256.0,66666.0)
#jonas_columns["ID"] = jonas_columns["ID"].replace(111509,111433)
#mistral_w5 = mistral_w5.append(jonas_columns, ignore_index=True)

In [143]:
mistral_w5["message_id"] = mistral_w5["message_id"].astype(int)
mistral_w5 = mistral_w5.dropna(subset=["message_id"])

In [144]:
mistral_wide = mistral_w5.pivot_table(
    index='Movez ID',
    columns='message_id',
    values='tst_message',
    aggfunc='last'  # or 'first', 'max', etc.
)

#Changing the column names of message ID & rename index
mistral_wide = mistral_wide.add_prefix('Q')
mistral_wide = mistral_wide.rename(columns={"message_id": "Participant ID"}, index={'message_id': 'Participant ID'})

#Sort columns according to their new value
q_columns = [col for col in mistral_wide.columns if col.startswith('Q')]
sorted_columns = sorted(q_columns, key=lambda x: int(x[1:]))


In [145]:
# Let's sort the columns according to the day they are shown
mistral_wide = mistral_wide[sorted_columns]

In [147]:
mistral_wide.head()

message_id,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12
Movez ID,,,,,,,,,,,,
0.0,"Hello! I’m Sasha, nice to meet you! Over the next two weeks, I will explore healthy and sustainable eating with you.\n\nA year ago, I started eating more plant-based foods and learned a lot along the way. It’s not about being perfect; I don’t think we all need to go completely vegan. However, being a bit mindful of how much animal food we eat and drink can be good for our health and the planet.\n\nStay tuned, as I will share lots of interesting science facts and give you tips for small changes that can make a big difference.","Fruits and vegetables are filled with important nutrients like vitamins, minerals, and antioxidants that help our bodies fight diseases. For instance, people who eat mostly plant foods were 39% less likely to get Covid and have fewer hospital visits overall.\n\nEvery time we buy food ourselves, we get to decide what goes into our bodies. To make it more fun, you could also talk with your friends during lunch and try out some plant-based foods together.","Reducing how much meat we eat can help save a lot of CO2, which causes climate change. For example, if we eat meat every other day instead of every day for a year, it saves as much CO2 as driving a car from London to China… and back!\n\nWith small changes like this, everyone can help the environment and slow down climate change now without waiting for others! Let’s start small; for example, we can try to eat meat only once a day, instead of having it at breakfast, lunch, and dinner.","More than 70% of farm animals in the UK are raised on factory farms to keep animal products cheap. On these farms, a chicken has only about the space of a laptop, so there's hardly any room to move. It's like living under a staircase, like Harry Potter, for your whole life.\n\nFactory farms keep going because people buy their products. By choosing to eat less meat or plant-based meats when we can, we can send a message to these farms to make better living conditions for their animals.","Fiber is really important for keeping our stomachs healthy, and it's only found in plants, not in animal foods. When we have enough fiber in our meals, we stay full for longer and have more energy throughout the school day, avoiding that tired feeling after lunch.\n\nChanging our habits, like what we eat, can be tricky at first. An easy way to start could be to try more plant-based lunches at school to feel more energetic and ready for the afternoon.","80% of young people believe that living sustainably is very important for their happiness. And 77% say they would choose meat alternatives as part of their sustainable lifestyle.\n\nIt's good to see how much our generation agrees on this. Even if we're not trying to be completely vegan, each of us can do something to help. For example, we can try the plant-based food options in our school cafeteria or some meat alternatives more often.","Hey! Thanks for following along for the first week.\n\nI'm excited to see how more people are choosing plant-based eating. Some of my favourite celebrities eat vegan, like Lewis Hamilton, Billie Eilish, and Ariana Grande. Others, like Harry Styles and Lionel Messi, avoid animal products for the most part.\n\nI also see new plant-based products and restaurants appearing everywhere. It looks like we'll have lots of options in the future!\n\nWhy not see if some of your favorite celebrities also eat plant-based?","Reducing how much meat, dairy, and sugary foods we eat can really help improve acne (up to 87% less acne in young people). Eating lots of vegetables and fruits also keeps our skin hydrated, protects us from getting sunburned, and even stops wrinkles.\n\nHaving clear, glowing skin might be as simple as just eating more veggies and fruits. When do you usually buy food during your day? Are there tasty plant-based options you could try the next time you have a choice?","If everyone ate less animal foods, the CO2 from food production 

In [148]:
# Last we need to replace the Movez ID key with the Avicenna ID key
# Create a dictionary mapping from 'Movez ID' to 'ID'
id_mapping = keys_w5.set_index('Movez ID')['ID'].to_dict()

# Replace the index of mistral_wide using the mapping
mistral_wide.index = mistral_wide.index.map(lambda idx: id_mapping.get(idx, idx))


In [149]:
# We rename the index to "ID"
mistral_wide.index.name = "ID"

In [150]:
mistral_index = list(mistral_wide.index)

In [151]:
# Lets have a final look into the data

mistral_wide.head()

message_id,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12
ID,,,,,,,,,,,,
115117.0,"Hello! I’m Sasha, nice to meet you! Over the next two weeks, I will explore healthy and sustainable eating with you.\n\nA year ago, I started eating more plant-based foods and learned a lot along the way. It’s not about being perfect; I don’t think we all need to go completely vegan. However, being a bit mindful of how much animal food we eat and drink can be good for our health and the planet.\n\nStay tuned, as I will share lots of interesting science facts and give you tips for small changes that can make a big difference.","Fruits and vegetables are filled with important nutrients like vitamins, minerals, and antioxidants that help our bodies fight diseases. For instance, people who eat mostly plant foods were 39% less likely to get Covid and have fewer hospital visits overall.\n\nEvery time we buy food ourselves, we get to decide what goes into our bodies. To make it more fun, you could also talk with your friends during lunch and try out some plant-based foods together.","Reducing how much meat we eat can help save a lot of CO2, which causes climate change. For example, if we eat meat every other day instead of every day for a year, it saves as much CO2 as driving a car from London to China… and back!\n\nWith small changes like this, everyone can help the environment and slow down climate change now without waiting for others! Let’s start small; for example, we can try to eat meat only once a day, instead of having it at breakfast, lunch, and dinner.","More than 70% of farm animals in the UK are raised on factory farms to keep animal products cheap. On these farms, a chicken has only about the space of a laptop, so there's hardly any room to move. It's like living under a staircase, like Harry Potter, for your whole life.\n\nFactory farms keep going because people buy their products. By choosing to eat less meat or plant-based meats when we can, we can send a message to these farms to make better living conditions for their animals.","Fiber is really important for keeping our stomachs healthy, and it's only found in plants, not in animal foods. When we have enough fiber in our meals, we stay full for longer and have more energy throughout the school day, avoiding that tired feeling after lunch.\n\nChanging our habits, like what we eat, can be tricky at first. An easy way to start could be to try more plant-based lunches at school to feel more energetic and ready for the afternoon.","80% of young people believe that living sustainably is very important for their happiness. And 77% say they would choose meat alternatives as part of their sustainable lifestyle.\n\nIt's good to see how much our generation agrees on this. Even if we're not trying to be completely vegan, each of us can do something to help. For example, we can try the plant-based food options in our school cafeteria or some meat alternatives more often.","Hey! Thanks for following along for the first week.\n\nI'm excited to see how more people are choosing plant-based eating. Some of my favourite celebrities eat vegan, like Lewis Hamilton, Billie Eilish, and Ariana Grande. Others, like Harry Styles and Lionel Messi, avoid animal products for the most part.\n\nI also see new plant-based products and restaurants appearing everywhere. It looks like we'll have lots of options in the future!\n\nWhy not see if some of your favorite celebrities also eat plant-based?","Reducing how much meat, dairy, and sugary foods we eat can really help improve acne (up to 87% less acne in young people). Eating lots of vegetables and fruits also keeps our skin hydrated, protects us from getting sunburned, and even stops wrinkles.\n\nHaving clear, glowing skin might be as simple as just eating more veggies and fruits. When do you usually buy food during your day? Are there tasty plant-based options you could try the next time you have a choice?","If everyone ate less animal foods, the CO2 from food production w

In [152]:
# Let's check if all the message cells have a text within them

for index, row in mistral_wide.iterrows():
    # Check for empty values in the row
    for column in mistral_wide.columns:
        if pd.isna(row[column]):  # Check if the value is NaN (empty)
            print(f"Empty value found in column '{column}' at row {index}")

In [157]:
# Assuming the index is a string type
invalid_indexes = mistral_wide.index[(mistral_wide.index < 100000.0) | (mistral_wide.index >= 200000.0)]

print("Indexes that are not five digits or do not start with '1':")
invalid_indexes

Indexes that are not five digits or do not start with '1':


Index([], dtype='float64', name='ID')

In [160]:
print(len(keys_w5))
print(len(mistral_wide))

286
278


In [161]:
missing_in_keys_w5 = keys_w5[~keys_w5["ID"].isin(mistral_wide.index)]

# Print these index values
print("Index values in df1 but not in keys_w4['ID']:")
missing_in_keys_w5

Index values in df1 but not in keys_w4['ID']:


,ID,Movez ID,Mail_child_exp,index,Norstat ID,Movez ID_y,condition
32,115532.0,87143.0,sammaccuirc@gmail.com,981.0,5YwxodQQ8X32Oylz,87143.0,1.0
39,115519.0,69968.0,kipkiruikoechrantich3@gmail.com,1959.0,nKY7owXjYk82G3v4,69968.0,2.0
67,115185.0,52999.0,qmazeedah@gmail.com,2168.0,gJdBAEy7naJAbz5P,52999.0,5.0
82,115120.0,42495.0,giftedcardiac@gmail.com,1779.0,LDen9KnOr1G2ElVw,75709.0,1.0
84,115114.0,77832.0,runicekhan@gmail.com,1960.0,p3MN234kmm19l1ag,77832.0,4.0
122,114887.0,80552.0,roangosling30@gmail.com,1539.0,GLPp2O5G7glA03ym,80552.0,3.0
134,114816.0,19626.0,marcuseriksen50@gmail.com,185.0,5nJ4ApEKnb4oG3kx,19626.0,1.0
189,115732.0,80552.0,scronin11@hotmail.com,1539.0,GLPp2O5G7glA03ym,NaN,3.0


In [162]:
keys_w5[keys_w5["Movez ID"].isin(missing_in_keys_w5["Movez ID"])]

,ID,Movez ID,Mail_child_exp,index,Norstat ID,Movez ID_y,condition
32,115532.0,87143.0,sammaccuirc@gmail.com,981.0,5YwxodQQ8X32Oylz,87143.0,1.0
39,115519.0,69968.0,kipkiruikoechrantich3@gmail.com,1959.0,nKY7owXjYk82G3v4,69968.0,2.0
67,115185.0,52999.0,qmazeedah@gmail.com,2168.0,gJdBAEy7naJAbz5P,52999.0,5.0
69,115181.0,87143.0,paul.oziegbe@gmail.com,1567.0,beaJAWwrzb3Avw16,87143.0,3.0
82,115120.0,42495.0,giftedcardiac@gmail.com,1779.0,LDen9KnOr1G2ElVw,75709.0,1.0
84,115114.0,77832.0,runicekhan@gmail.com,1960.0,p3MN234kmm19l1ag,77832.0,4.0
122,114887.0,80552.0,roangosling30@gmail.com,1539.0,GLPp2O5G7glA03ym,80552.0,3.0
134,114816.0,19626.0,marcuseriksen50@gmail.com,185.0,5nJ4ApEKnb4oG3kx,19626.0,1.0
189,115732.0,80552.0,scronin11@hotmail.com,1539.0,GLPp2O5G7glA03ym,NaN,3.0
190,115730.0,52999.0,omoba.quadri12@gmail.com,2168.0,gJdBAEy7naJAbz5P,NaN,5.0


In [193]:
#mistral_wide.to_csv(MESSAGE_DIR / 'wave5' / 'messages_w5_new.csv', encoding='utf-8', index=True)